## I/O: Reading and Writing Data

The most standard use of pandas is to read and write data files. Pandas provides a series of [built-in I/O functions](https://pandas.pydata.org/pandas-docs/stable/user_guide/io.html) to read and write data from various files formats; making it the defacto standard tool to convert files formats.

Pandas is often used to read data from basic internet and SQL files formats such as CSVs and Json files and transform them into **Big Data** formats such as **Parguet, ORC, BigQuery**, and other formats.

### Reading CSV

Amongst pandas [built-in readers](https://pandas.pydata.org/pandas-docs/stable/user_guide/io.html)you can use `read_csv` to import data from a dleimited file:

:::tip `flights.csv` 

Data files for this lesson are included under `data/` forlder. The `flights.csv` files contains all domestic (USA) flights for 2019 Thanksgiving Day. These are real flight records from United States Beauru of Transportation.

:::

<br/>


In [1]:
import pandas as pd

flights = pd.read_csv('../data/flights.csv', header=0)
print(flights.head())

  flight_date airline tailnumber  flight_number  src dest  departure_time  \
0  2019-11-28      9E     N8974C           3280  CHA  DTW            1300   
1  2019-11-28      9E     N901XJ           3281  JAX  RDU             700   
2  2019-11-28      9E     N901XJ           3282  RDU  LGA             900   
3  2019-11-28      9E     N912XJ           3283  DTW  ATW            1216   
4  2019-11-28      9E     N924XJ           3284  DSM  MSP            1103   

   arrival_time  flight_time  distance  
0          1455        115.0     505.0  
1           824         84.0     407.0  
2          1039         99.0     431.0  
3          1242         86.0     296.0  
4          1211         68.0     232.0  


`read_csv` methods provides a series of options to parse csv files correctly. The `header` option is used to extract column names from a csv header row. `header=0` marks the first row of csv (row 0) as the header row.

Feel free to set other options:

In [2]:
import pandas as pd

# setting separator and line terminator characters
flights = pd.read_csv('../data/flights.csv', header=0, sep=',', lineterminator='\n')

# reading only 10 rows and selected columns
flights = pd.read_csv('../data/flights.csv', header=0, nrows=10, 
                      usecols=['airline', 'src', 'dest'])

# print
print(flights.head())

  airline  src dest
0      9E  CHA  DTW
1      9E  JAX  RDU
2      9E  RDU  LGA
3      9E  DTW  ATW
4      9E  DSM  MSP


:::info `read_csv` options

For the full list of available `read_csv` options refer to the online 
[documentation](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.read_csv.html)

:::

<br/>

### Assigning data types

You can set column data types using the `dtype` option:

In [4]:
import pandas as pd
import numpy as np

# using `dtype` to assign particular column data types
flights = pd.read_csv('../data/flights.csv', header=0,
                      dtype={
                          'flight_time': np.int16,
                          'distance': np.int16
                      })

# print
print(flights.head(10))

  flight_date airline tailnumber  flight_number  src dest  departure_time  \
0  2019-11-28      9E     N8974C           3280  CHA  DTW            1300   
1  2019-11-28      9E     N901XJ           3281  JAX  RDU             700   
2  2019-11-28      9E     N901XJ           3282  RDU  LGA             900   
3  2019-11-28      9E     N912XJ           3283  DTW  ATW            1216   
4  2019-11-28      9E     N924XJ           3284  DSM  MSP            1103   
5  2019-11-28      9E     N833AY           3285  LGA  PWM            1013   
6  2019-11-28      9E     N314PQ           3286  CLE  DTW            1400   
7  2019-11-28      9E     N686BR           3288  DTW  LAN            1227   
8  2019-11-28      9E     N686BR           3288  LAN  DTW            1350   
9  2019-11-28      9E     N147PQ           3289  JFK  ROC            2100   

   arrival_time  flight_time  distance  
0          1455          115       505  
1           824           84       407  
2          1039           99 

Data types are typically set as numpy types. The dtype parameter is specifically handy since it allows you to set specific columns and leave the rest for pandas to figure out.

### Using Converters

The most convinient way to parse special columns and apply business rules to transform fields at ingest is using the `converters` option of `read_csv`.

You can use specific function to parse special fields. In this case we use a couple functions called `decode_flightdate` and `decode_tailnumber` to parse flight dates and drop the initial letter 'N' from tailnumber. We also show that you can use `lambda` functions as converters:

<br/>


In [5]:
import pandas as pd
from datetime import datetime

def decode_flightdate(value:str):
    try:
        return datetime.strptime(value, '%Y-%m-%d').date()
    except (ValueError, TypeError):
        return None

def decode_tailnumber(value:str):
    if str(value).startswith('N'):
        return str(value)[1:]
    else:
        return str(value)

# using `converters` to pass functions to parse fields
flights = pd.read_csv('../data/flights.csv', header=0,
                      converters={
                          'flight_time': decode_flightdate,
                          'tailnumber': decode_tailnumber,
                          'flight_time': (lambda v: int(float(v))),
                          'distance': (lambda v: int(float(v))),
                      })

# print
print(flights.head(10))

  flight_date airline tailnumber  flight_number  src dest  departure_time  \
0  2019-11-28      9E      8974C           3280  CHA  DTW            1300   
1  2019-11-28      9E      901XJ           3281  JAX  RDU             700   
2  2019-11-28      9E      901XJ           3282  RDU  LGA             900   
3  2019-11-28      9E      912XJ           3283  DTW  ATW            1216   
4  2019-11-28      9E      924XJ           3284  DSM  MSP            1103   
5  2019-11-28      9E      833AY           3285  LGA  PWM            1013   
6  2019-11-28      9E      314PQ           3286  CLE  DTW            1400   
7  2019-11-28      9E      686BR           3288  DTW  LAN            1227   
8  2019-11-28      9E      686BR           3288  LAN  DTW            1350   
9  2019-11-28      9E      147PQ           3289  JFK  ROC            2100   

   arrival_time  flight_time  distance  
0          1455          115       505  
1           824           84       407  
2          1039           99 

:::tip Using `converters` functoins

We highly recommend using the converter funtions for parsing and applying business rules and cleansing rules at parse time with read_csv.

:::

### Writing Data

Pandas provides a series of [I/O write funtions](https://pandas.pydata.org/pandas-docs/stable/user_guide/io.html). You can read the documentaion to use appropriate function for your use-case. 

Here we're going to write our flights into both **Json Row** and **Parquet** formats:


In [6]:
import pandas as pd

# read csv
flights = pd.read_csv('../data/flights.csv', header=0)

# write json row format
flights.to_json('../data/flights.json', orient='records', lines=True)
# write compressed parquet format
flights.to_parquet('../data/flights.parquet', engine='pyarrow', 
                   compression='gzip', index=False)

print('done.')

done.
